**Оконные функции**

Примените оконные функции к таблице products и с помощью ранжирующих функций упорядочьте все товары по цене — от самых дорогих к самым дешёвым. Добавьте в таблицу следующие колонки:
Колонку product_number с порядковым номером товара (функция ROW_NUMBER).
Колонку product_rank с рангом товара с пропусками рангов (функция RANK).
Колонку product_dense_rank с рангом товара без пропусков рангов (функция DENSE_RANK).


In [ ]:
SELECT 
    product_id,
    name,
    price,
    ROW_NUMBER() OVER (ORDER BY price DESC) AS product_number,
    RANK() OVER (ORDER BY price DESC) AS product_rank,
    DENSE_RANK() OVER (ORDER BY price DESC) AS product_dense_rank
FROM 
    products

Примените оконную функцию к таблице products и с помощью агрегирующей функции в отдельной колонке для каждой записи проставьте цену самого дорогого товара. Колонку с этим значением назовите max_price.
Затем для каждого товара посчитайте долю его цены в стоимости самого дорогого товара — просто поделите одну колонку на другую. Полученные доли округлите до двух знаков после запятой. Колонку с долями назовите share_of_max.
Выведите всю информацию о товарах, включая значения в новых колонках. Результат отсортируйте сначала по убыванию цены товара, затем по возрастанию id товара.

In [ ]:
SELECT
    product_id,
    name,
    price,
    MAX(price) OVER() AS max_price,
    ROUND((price / MAX(price) OVER()), 2) AS share_of_max
FROM 
    products
GROUP BY
    product_id
ORDER BY
    price DESC,
    product_id

Примените две оконные функции к таблице products. Одну с агрегирующей функцией MAX, а другую с агрегирующей функцией MIN — для вычисления максимальной и минимальной цены. Для двух окон задайте инструкцию ORDER BY по убыванию цены. Поместите результат вычислений в две колонки max_price и min_price.
Выведите всю информацию о товарах, включая значения в новых колонках. Результат отсортируйте сначала по убыванию цены товара, затем по возрастанию id товара.

In [ ]:
SELECT
    product_id,
    name,
    price,
    MAX(price) OVER(ORDER BY price DESC) AS max_price,
    MIN(price) OVER(ORDER BY price DESC) AS min_price
FROM 
    products
GROUP BY
    product_id
ORDER BY
    price DESC,
    product_id

Сначала на основе таблицы orders сформируйте запрос, который вернет таблицу с общим числом заказов по дням. При подсчёте числа заказов не учитывайте отменённые заказы (их можно определить по таблице user_actions). Колонку с днями назовите date, а колонку с числом заказов — orders_count.
Затем поместите полученную таблицу в подзапрос и примените к ней оконную функцию в паре с агрегирующей функцией SUM для расчёта накопительной суммы числа заказов. Не забудьте для окна задать инструкцию ORDER BY по дате.
Колонку с накопительной суммой назовите orders_count_cumulative. 

In [ ]:
WITH 
cancel_orders AS (
    SELECT 
        order_id
    FROM 
        user_actions
    WHERE
        action = 'cancel_order'
),
order_days as (
    SELECT 
        COUNT(order_id) as orders_count,
        creation_time::date as date
    from 
        orders
    where order_id NOT IN (select order_id from cancel_orders)
    GROUP BY
        date
)
select 
    date,
    orders_count,
    sum(orders_count) OVER(order by date)::int AS orders_count_cumulative
from 
    order_days
ORDER BY 
    date

Для каждого пользователя в таблице user_actions посчитайте порядковый номер каждого заказа.

In [ ]:
WITH
cancel_order AS (
    SELECT
        order_id
    FROM 
        user_actions
    WHERE
        action = 'cancel_order'
)
SELECT 
    user_id,
    order_id,
    time,
    ROW_NUMBER() OVER(PARTITION BY user_id ORDER BY time) AS order_number 
FROM
    user_actions
WHERE
    order_id NOT IN (SELECT order_id FROM cancel_order)
ORDER BY 
    user_id,
    order_number
LIMIT 
    1000

Дополните запрос из предыдущего задания и с помощью оконной функции для каждого заказа каждого пользователя рассчитайте, сколько времени прошло с момента предыдущего заказа. 

In [ ]:
WITH
cancel_order AS (
    SELECT
        order_id
    FROM 
        user_actions
    WHERE
        action = 'cancel_order'
)
select 
    user_id,
    order_id,
    time,
    ROW_NUMBER() OVER(PARTITION BY user_id ORDER BY time) AS order_number,
    LAG(time, 1) OVER(PARTITION BY user_id ORDER BY time) AS time_lag,
    (time - (LAG(time, 1) OVER(PARTITION BY user_id ORDER BY time))) AS time_diff 
FROM
    user_actions
WHERE
    order_id NOT IN (SELECT order_id FROM cancel_order)
ORDER BY 
    user_id,
    order_number
LIMIT 
    1000

На основе запроса из предыдущего задания для каждого пользователя рассчитайте, сколько в среднем времени проходит между его заказами. Посчитайте этот показатель только для тех пользователей, которые за всё время оформили более одного неотмененного заказа.

In [ ]:
WITH
canceled_order AS (
    SELECT
        order_id
    FROM 
        user_actions
    WHERE
        action = 'cancel_order'
),
user_order_times AS (
select 
    user_id,
    COUNT(order_id) OVER (PARTITION BY user_id) AS user_order_count,
    time,
    (time - (LAG(time, 1) OVER(PARTITION BY user_id ORDER BY time))) AS time_diff 
FROM
    user_actions
WHERE
    order_id NOT IN (SELECT order_id FROM canceled_order)
)
SELECT 
    user_id,
    (EXTRACT(EPOCH from AVG(time_diff))/3600)::int AS hours_between_orders
FROM
	user_order_times
WHERE
	user_order_count > 1
GROUP BY
	user_id
ORDER BY
	user_id
LIMIT
	1000

Сначала на основе таблицы orders сформируйте новую таблицу с общим числом заказов по дням. Вы уже делали это в одной из предыдущих задач. При подсчёте числа заказов не учитывайте отменённые заказы (их можно определить по таблице user_actions). Колонку с числом заказов назовите orders_count.
Затем поместите полученную таблицу в подзапрос и примените к ней оконную функцию в паре с агрегирующей функцией AVG для расчёта скользящего среднего числа заказов. Скользящее среднее для каждой записи считайте по трём предыдущим дням. Подумайте, как правильно задать границы рамки, чтобы получить корректные расчёты.
Полученные значения скользящего среднего округлите до двух знаков после запятой. Колонку с рассчитанным показателем назовите moving_avg. Сортировку результирующей таблицы делать не нужно.

In [ ]:
WITH
canceled_orders AS (
    SELECT
        order_id
    FROM
        user_actions
    WHERE
        action = 'cancel_order'
),
daily_orders AS (
    SELECT 
        COUNT(order_id) AS orders_count,
        creation_time::date AS date
    FROM 
        orders
    WHERE
        order_id NOT IN (SELECT order_id FROM canceled_orders)
    GROUP BY
        date
)
SELECT
    orders_count,
    date,
    ROUND(AVG(orders_count) OVER(ORDER BY date ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING), 2) AS moving_avg
FROM daily_orders

Отметьте в отдельной таблице тех курьеров, которые доставили в сентябре 2022 года заказов больше, чем в среднем все курьеры.

In [ ]:
SELECT
    courier_id,
    COUNT(order_id) AS delivered_orders,
    ROUND(AVG(COUNT(order_id)) OVER(),2) AS avg_delivered_orders,
    CASE
        WHEN COUNT(order_id) > ROUND(AVG(COUNT(order_id)) OVER(),2) THEN 1
        ELSE 0
    END AS is_above_avg
FROM
    courier_actions
WHERE 
    action = 'deliver_order' 
    AND DATE_PART('year', time) = 2022 
    AND DATE_PART('month', time) = 9 
GROUP BY
    courier_id

По данным таблицы user_actions посчитайте число первых и повторных заказов на каждую дату.

In [ ]:
WITH
cancel_ordered AS (
    SELECT 
        order_id
    FROM
        user_actions
    WHERE
        action = 'cancel_order'
),
orders_with_types AS (
    SELECT 
        time::date AS date,
        order_id,
        CASE
            WHEN time = MIN(time) OVER(PARTITION BY user_id) THEN 'Первый'
            ELSE 'Повторный'
        END AS order_type
    FROM
        user_actions
    WHERE
        order_id NOT IN (SELECT order_id FROM cancel_ordered)
)
SELECT
    date,
    order_type,
    COUNT(order_id) AS orders_count
FROM
    orders_with_types
GROUP BY
    date,
    order_type
ORDER BY
    date,
    order_type

К запросу, полученному на предыдущем шаге, примените оконную функцию и для каждого дня посчитайте долю первых и повторных заказов. Сохраните структуру полученной ранее таблицы и добавьте только одну новую колонку с посчитанными значениями.

In [ ]:
WITH
cancel_ordered AS (
    SELECT 
        order_id
    FROM
        user_actions
    WHERE
        action = 'cancel_order'
),
orders_with_types AS (
    SELECT 
        time::date AS date,
        order_id,
        CASE
            WHEN time = MIN(time) OVER(PARTITION BY user_id) THEN 'Первый'
            ELSE 'Повторный'
        END AS order_type
    FROM
        user_actions
    WHERE
        order_id NOT IN (SELECT order_id FROM cancel_ordered)
),
daily_counts AS (
    SELECT
        date,
        order_type,
        COUNT(order_id) AS orders_count,
        SUM(COUNT(order_id)) OVER(PARTITION BY date) AS total_orders
    FROM
        orders_with_types
    GROUP BY
        date,
        order_type
)
SELECT
    date, 
    order_type,
    orders_count, 
    ROUND((orders_count / total_orders), 2) AS orders_share
FROM
    daily_counts
ORDER BY
    date,
    order_type

Примените оконную функцию к таблице products и с помощью агрегирующей функции в отдельной колонке для каждой записи проставьте среднюю цену всех товаров. Колонку с этим значением назовите avg_price.

In [ ]:
WITH
max_prices AS (
    SELECT
        MAX(price) AS max_price
    FROM
        products
)
SELECT
    product_id,
    name,
    price,
    ROUND(AVG(price) OVER(), 2) AS avg_price,
    ROUND(AVG(price) FILTER(WHERE price != (SELECT max_price FROM max_prices)) OVER(), 2) AS avg_price_filtered
FROM
    products
ORDER BY
    price DESC,
    product_id 

Для каждой записи в таблице user_actions с помощью оконных функций и предложения FILTER посчитайте, сколько заказов сделал и сколько отменил каждый пользователь на момент совершения нового действия.

In [ ]:
WITH
user_order_stats AS (
    SELECT
        user_id,
        order_id,
        action,
        time,
        COUNT(order_id) FILTER(WHERE action = 'create_order') OVER(PARTITION BY user_id ORDER BY time ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS created_orders,
        COUNT(order_id) FILTER(WHERE action = 'cancel_order') OVER(PARTITION BY user_id ORDER BY time ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS canceled_orders
    FROM
        user_actions
)
SELECT
    user_id,
    order_id,
    action, 
    time, 
    created_orders,
    canceled_orders,
    CASE 
        WHEN created_orders > 0 THEN ROUND(canceled_orders::numeric / created_orders, 2)
        ELSE NULL
    END AS cancel_rate
FROM
    user_order_stats
ORDER BY
    user_id,
    order_id,
    time
LIMIT
    1000

Из таблицы courier_actions отберите топ 10% курьеров по количеству доставленных за всё время заказов. Выведите id курьеров, количество доставленных заказов и порядковый номер курьера в соответствии с числом доставленных заказов.
У курьера, доставившего наибольшее число заказов, порядковый номер должен быть равен 1, а у курьера с наименьшим числом заказов — числу, равному десяти процентам от общего количества курьеров в таблице courier_actions.

In [ ]:
WITH
courier_count AS (
    SELECT
        COUNT(DISTINCT courier_id) AS total_couriers
    FROM courier_actions
),
courier_stats AS (
    SELECT 
        courier_id,
        COUNT(DISTINCT (order_id)) AS orders_count,
        ROW_NUMBER() OVER(ORDER BY COUNT(DISTINCT(order_id)) DESC, courier_id) AS courier_rank,
        (SELECT total_couriers FROM courier_count) AS total,
        CEIL((SELECT total_couriers FROM courier_count) * 0.1) AS top_10_percent
    FROM
        courier_actions
    WHERE
        action = 'deliver_order'
    GROUP BY
        courier_id
)
SELECT
    courier_id,
    orders_count,
    courier_rank
FROM
    courier_stats
WHERE
    courier_rank <= top_10_percent
ORDER BY
    courier_rank

С помощью оконной функции отберите из таблицы courier_actions всех курьеров, которые работают в нашей компании 10 и более дней. Также рассчитайте, сколько заказов они уже успели доставить за всё время работы.

In [ ]:
WITH
courier_stats AS (
    SELECT 
        courier_id,
        COUNT(DISTINCT(order_id)) FILTER (WHERE action = 'deliver_order') AS delivered_orders,
        MAX(time) AS max_time,
        MIN(time) AS min_time
    FROM
        courier_actions 
    GROUP BY
        courier_id
),
courier_employment AS (
    SELECT
        courier_id,
        delivered_orders,
        DATE_PART('days', MAX(max_time) OVER() - min_time)::integer AS days_employed
    FROM
        courier_stats
)
SELECT
    courier_id, 
    days_employed, 
    delivered_orders
FROM
    courier_employment
WHERE
    days_employed >= 10
ORDER BY
    days_employed DESC,
    courier_id

На основе информации в таблицах orders и products рассчитайте стоимость каждого заказа, ежедневную выручку сервиса и долю стоимости каждого заказа в ежедневной выручке, выраженную в процентах.

In [ ]:
WITH
canceled_orders AS (
    SELECT
        order_id
    FROM
        user_actions
    WHERE
        action = 'cancel_order'
),
order_products AS (
    SELECT 
        order_id,
        creation_time AS creation_time,
        UNNEST(product_ids) AS product_id
    FROM
        orders
    WHERE
        order_id NOT IN (SELECT order_id FROM canceled_orders)
),
order_prices AS (
    SELECT
        O.order_id AS order_id,
        O.creation_time AS creation_time,
        SUM(P.price) AS order_price
    FROM
        order_products AS O
        LEFT JOIN products AS P
        ON O.product_id = P.product_id
    GROUP BY
        O.order_id,
        O.creation_time
),
daily_revenues AS (
    SELECT
        DATE(creation_time) AS order_date,
        SUM(order_price) AS daily_revenue
    FROM
        order_price
    GROUP BY
        DATE(creation_time)
)
SELECT
    OP.order_id AS order_id,
    OP.creation_time AS creation_time,
    OP.order_price AS order_price,
    DR.daily_revenue AS daily_revenue,
    ROUND((100 * OP.order_price::DECIMAL / DR.daily_revenue), 3) AS percentage_of_daily_revenue
FROM
    order_price OP
    JOIN daily_revenues DR
    ON DATE(OP.creation_time) = DR.order_date
ORDER BY
    DATE(OP.creation_time) DESC,
    percentage_of_daily_revenue DESC,
    OP.order_id

На основе информации в таблицах orders и products рассчитайте ежедневную выручку сервиса и отразите её в колонке daily_revenue. Затем с помощью оконных функций и функций смещения посчитайте ежедневный прирост выручки. Прирост выручки отразите как в абсолютных значениях, так и в % относительно предыдущего дня. Колонку с абсолютным приростом назовите revenue_growth_abs, а колонку с относительным — revenue_growth_percentage.

In [ ]:
WITH 
canceled_orders AS (
    SELECT 
        order_id
    FROM 
        user_actions
    WHERE 
        action = 'cancel_order'
),
order_products AS (
    SELECT 
        order_id,
        creation_time,
        UNNEST(product_ids) AS product_id
    FROM 
        orders
    WHERE 
        order_id NOT IN (SELECT order_id FROM canceled_orders)
),
daily_revenues AS (
    SELECT 
        DATE(O.creation_time) AS date,
        SUM(P.price) AS daily_revenue
    FROM 
        order_products AS O
        LEFT JOIN products AS P
        ON O.product_id = P.product_id
    GROUP BY 
        DATE(creation_time)
)
SELECT 
    date,
    ROUND(daily_revenue, 1) AS daily_revenue,
    ROUND(COALESCE(daily_revenue - LAG(daily_revenue, 1) OVER (ORDER BY date), 0), 1) AS revenue_growth_abs,
    ROUND(COALESCE(ROUND((daily_revenue - LAG(daily_revenue, 1) OVER (ORDER BY date))::DECIMAL / NULLIF(LAG(daily_revenue, 1) OVER (ORDER BY date), 0) * 100, 2), 0), 1) AS revenue_growth_percentage
FROM 
    daily_revenues
ORDER BY 
    date

С помощью оконной функции рассчитайте медианную стоимость всех заказов из таблицы orders, оформленных в нашем сервисе. В качестве результата выведите одно число. Колонку с ним назовите median_price. Отменённые заказы не учитывайте.

In [ ]:
WITH
canceled_orders AS (
    SELECT 
        order_id
    FROM
        user_actions
    WHERE
        action = 'cancel_order'
),
order_products AS (
    SELECT
        order_id,
        UNNEST(product_ids) AS product_id
    FROM
        orders
    WHERE
        order_id NOT IN (SELECT order_id FROM canceled_orders)
),
order_prices AS (
    SELECT
        OP.order_id AS order_id,
        SUM(P.price) AS order_price
    FROM
        order_products AS OP
        LEFT JOIN products AS P
        ON OP.product_id = P.product_id
    GROUP BY
        OP.order_id
),
ranked_orders AS (
    SELECT 
        order_price,
        ROW_NUMBER() OVER(ORDER BY order_price) AS row_number,
        COUNT(*) OVER() AS total_rows
    FROM
    order_prices    
)
SELECT
    AVG(order_price) AS median_price
FROM 
    ranked_orders
WHERE
    row_number BETWEEN total_rows / 2.0 AND total_rows / 2.0 + 1 